## looks like more graph theory

In [1]:
from util import file_to_list
from math import sqrt

In [2]:
def to_coords(raw):
    coords = []
    for line in raw:
        num = [int(x) for x in line.split(',')]
        coords.append(num)
    return coords

In [3]:
def dist(a, b):
    x = a[0] - b[0]
    y = a[1] - b[1]
    z = a[2] - b[2]
    return sqrt(x**2 + y**2 + z**2)

In [4]:
def pairwise(boxes):
    N = len(boxes)
    distances = []
    for i in range(N-1):
        for j in range(i+1, N):
            distances.append([dist(boxes[i], boxes[j]), i, j])
    distances.sort()
    return distances

In [5]:
def update(circuits, old, new):
    for i, c in enumerate(circuits):
        if c == old:
            circuits[i] = new

In [6]:
def connect(boxes, distances, top=10):
    circuits = [-1] * len(boxes)
    added = 0
    current_circuit = 0

    for _, i, j in distances:
        # print(f'comparing boxes {i, j}...')
        # if no box belongs to a circuit
        if circuits[i] == circuits[j] == -1:
            # print(f'- both unassigned; opening circuit no. {current_circuit}')
            circuits[i] = current_circuit
            circuits[j] = current_circuit
            current_circuit += 1
            added += 1
        # if box j belongs to a circuit
        elif circuits[i] == -1 and circuits[j] > -1:
            # print(f'- assigning box {i} to circuit {circuits[j]}')
            circuits[i] = circuits[j]
            added += 1
        elif circuits[i] > -1 and circuits[j] == -1:
            # print(f'- assigning box {j} to circuit {circuits[i]}')
            circuits[j] = circuits[i]
            added += 1
        elif circuits[i] == circuits[j] and circuits[j] > -1:
            # print(f'- both boxes in same circuit')
            pass
        elif circuits[i] > -1 and circuits[j] > -1:
            # print(f'- both boxes in circuits, updating from {circuits[i]} to {circuits[j]}:')
            update(circuits, circuits[i], circuits[j])
            # print(circuits)
        # print(f'= have added {added} cables so far')
        if added == top:
            return circuits

    return None

In [7]:
def count(circuits):
    freq = []
    for c in set(circuits):
        if c > -1:
            freq.append([circuits.count(c), c])
    return freq

def top3(counts):
    counts.sort(reverse=True)
    res = 1
    for i in range(3):
        res *= counts[i][0]
    return res

In [8]:
raw = file_to_list('./data/day08_test.txt')
boxes = to_coords(raw)
distances = pairwise(boxes)
circuits = connect(boxes, distances)
counts = count(circuits)
top3(counts)

50

In [ ]:
raw = file_to_list('./data/day08.txt')
boxes = to_coords(raw)
distances = pairwise(boxes)
circuits = connect(boxes, distances, top=1000)
counts = count(circuits)
top3(counts)